In [4]:
# Подключение Google Диска (для доступа к файлу с датасетом)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Импорт необходимых библиотек из PySpark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, regexp_extract
from pyspark.sql.types import DoubleType

In [9]:
# Создание сессии Spark
spark # Загрузка CSV-файла с треками из Google Drive
df = spark.read.option("header", True).option("inferSchema", True).csv(
    "/content/drive/MyDrive/tracks.csv"
) = SparkSession.builder.appName("SpotifyTrends").getOrCreate()

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/drive/MyDrive/tracks.csv.

In [ ]:
# Извлечение года из поля release_date, поддерживаются разные форматы:
# yyyy, yyyy-MM, yyyy-MM-dd, dd/MM/yyyy
df = df.withColumn(
    "release_year",
    when(col("release_date").rlike(r"^\d{4}$"),
         col("release_date").cast("int"))
    .when(col("release_date").rlike(r"^\d{4}-\d{2}$"),
          regexp_extract(col("release_date"), r"^(\d{4})", 1).cast("int"))
    .when(col("release_date").rlike(r"^\d{2}/\d{2}/\d{4}$"),
          regexp_extract(col("release_date"), r"(\d{4})$", 1).cast("int"))
    .when(col("release_date").rlike(r"^\d{4}-\d{2}-\d{2}$"),
          regexp_extract(col("release_date"), r"^(\d{4})", 1).cast("int"))
    .otherwise(None)
)

In [ ]:
# Фильтрация строк с корректным годом выпуска
df = df.filter(col("release_year").isNotNull())

In [ ]:
# Оставляем только треки, выпущенные с 1920 года
df = df.filter(col("release_year") >= 1920)

In [ ]:
# Вычисление десятилетия для каждого трека (например, 1987 → 1980)
df = df.withColumn("decade", (col("release_year") / 10).cast("int") * 10)

In [ ]:
# Список аудиопараметров, которые будут анализироваться
features = [
    "danceability", "energy", "loudness", "speechiness",
    "acousticness", "instrumentalness", "liveness", "valence", "tempo"
]

In [ ]:
# Приведение каждого параметра к типу Double (для последующей агрегации)
for feature in features:
    df = df.withColumn(feature, col(feature).cast(DoubleType()))

In [ ]:
# Группировка по десятилетиям и вычисление среднего значения для каждого параметра
agg_df = df.groupBy("decade").avg(*features).orderBy("decade")

In [ ]:
# Сохранение результата в CSV-файл (в одну партицию, с заголовками)
agg_df.coalesce(1).write.option("header", True).mode("overwrite").csv("/content/drive/MyDrive/output/spotify_trends")

In [ ]:
# Вывод полученной таблицы в консоль
agg_df.show()

In [ ]:
# Завершение сессии Spark
spark.stop()